## Initialisation and Training of ANN-4

In [ ]:
# public modules 
import numpy as np
from tensorflow import keras
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Input, Model
import tensorflow as tf

In [ ]:
# check to see if GPU is in use
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# connection to google drive + import data
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/content")

# importing Heston data
data = np.genfromtxt('heston_data.csv', delimiter=',')

# inputs: converting spot price + strike price to moneyness (s/k)
inputs = data[:,:9] 
X = np.array([np.array([x[0]/x[1], x[2],x[3],x[4], x[5], x[6], x[7], x[8]])for x in inputs])

# outputs: scaling the prices byt the strike price (p/k)
y_calls = np.array(data[:,-2]) / data[:,1]
y_puts = np.array(data[:,-1]) / data[:,1]

# training data
X_train = X[:150000]
y_calls_train = y_calls[:150000]
y_puts_train = y_puts[:150000]

# validation data
X_val = X[150000:175000]
y_calls_val = y_calls[150000:175000]
y_puts_val = y_puts[150000:175000]

# testing data
X_test = X[175000:]
y_calls_test = y_calls[175000:]
y_puts_test = y_puts[175000:]

print(X_train.shape)
print(y_calls_train.shape)
print(y_puts_train.shape)
print(X_val.shape)
print(y_calls_val.shape)
print(y_puts_val.shape)
print(X_test.shape)
print(y_calls_test.shape)
print(y_puts_test.shape)

Mounted at /content/drive
(150000, 8)
(150000,)
(150000,)
(25000, 8)
(25000,)
(25000,)
(25000, 8)
(25000,)
(25000,)


In [ ]:
# ANN-4 model initialisation
def build_model_optimal():
  inputs = Input(shape=(8,), name='input')

  activations = 'relu'

  dropout = False

  x = inputs
  for i in range(9):
    x = Dense(units=500,
              kernel_initializer='glorot_uniform',
              activation = activations)(x)
    if dropout == True:
      x = Dropout(0.2)(x)
     
  call = Dense(1, name='call_output')(x)
  put = Dense(1, name='put_output')(x)

  opt = keras.optimizers.Adam(1e-5)

  model = Model(inputs=inputs, outputs=[call, put])
  model.compile(loss='mse', optimizer=opt)

  return model

ann_4 = build_model_optimal()

# ANN-4 training
ann_4.fit(X_train, {'call_output': y_calls_train, 'put_output': y_puts_train},
                validation_data = (X_val, {'call_output': y_calls_val, 'put_output': y_puts_val}),
                epochs=200, batch_size=200)

Epoch 1/200
750/750 [==============================] - 3s 4ms/step - loss: 0.0415 - call_output_loss: 0.0346 - put_output_loss: 0.0069 - val_loss: 0.0020 - val_call_output_loss: 0.0013 - val_put_output_loss: 7.2530e-04
Epoch 2/200
750/750 [==============================] - 3s 4ms/step - loss: 6.0152e-04 - call_output_loss: 3.5948e-04 - put_output_loss: 2.4204e-04 - val_loss: 1.2748e-04 - val_call_output_loss: 6.0312e-05 - val_put_output_loss: 6.7163e-05
Epoch 3/200
750/750 [==============================] - 3s 3ms/step - loss: 9.4399e-05 - call_output_loss: 4.7419e-05 - put_output_loss: 4.6980e-05 - val_loss: 8.1813e-05 - val_call_output_loss: 4.3861e-05 - val_put_output_loss: 3.7952e-05
Epoch 4/200
750/750 [==============================] - 3s 3ms/step - loss: 6.7242e-05 - call_output_loss: 3.4816e-05 - put_output_loss: 3.2426e-05 - val_loss: 5.9347e-05 - val_call_output_loss: 3.0343e-05 - val_put_output_loss: 2.9004e-05
Epoch 5/200
750/750 [==============================] - 3s 3ms/st

In [ ]:
# saving ANN-4
ann_4.save('ANN-4')

INFO:tensorflow:Assets written to: ANN-4_v2/assets
